winget install 'Microsoft .NET SDK 7.0'  #Needed. Doesn't work on .Net 8
### Health Check

In [1]:
[ordered]@{"$((Get-TimeZone).Id)" = Get-Date
    'Boot'      = (Get-CimInstance win32_operatingsystem).lastbootuptime.ToString('ddd M.d.yy H:m')
    'CpuUsage%' = (Get-CimInstance win32_processor | Measure-Object LoadPercentage -Average).Average
    'RamUsage%' = {$os=Get-CimInstance win32_operatingsystem; [Math]::Ceiling(($os.TotalVisibleMemorySize-$os.FreePhysicalMemory)*100/$os.TotalVisibleMemorySize)}.Invoke()
    'C:%Free'   = { $disk = Get-CimInstance Win32_LogicalDisk -Filter "DeviceID='C:'"; [Math]::Ceiling(($disk.FreeSpace / $disk.Size) * 100) }.Invoke()
            } | ft -HideTableHeaders -AutoSize


Central Standard Time 12/2/2023 9:00:01 PM
Boot                  Sat 12.2.23 10:36
CpuUsage%             76
RamUsage%             {51}
C:%Free               {55}



This shows which processes are consuming the most cpu, memory and network and have the most number of handles and threads currently on the system.

In [12]:
$objArr = Get-Process | Select-Object @{Name='Process'; Expression={$_.ProcessName+'►'+$_.Id}}, CPU, ID, WS, HandleCount, Threads

$Network = (Get-Counter '\Process(*)\IO Read Operations/sec').CounterSamples.Where({$_.InstanceName -ne "_Total" -and $_.InstanceName -ne "Idle"})|
                sort cookedvalue -Descending|Select -Property instancename -First 10

# Sort for each property separately
$sortedByCPU = $objArr | Sort-Object CPU -Descending | Select-Object -First 10
$sortedByWS = $objArr | Sort-Object WS -Descending | Select-Object -First 10
$sortedByHandles = $objArr | Sort-Object HandleCount -Descending | Select-Object -First 10
$sortedByThreads = $objArr | Sort-Object Threads -Descending | Select-Object -First 10

$harray = for ($j = 0; $j -lt 10; $j++) {
    [PSCustomObject][ordered]@{
        'CPU'     = $sortedByCPU[$j].Process
        'WS'      = $sortedByWS[$j].Process
        'Handles' = $sortedByHandles[$j].Process
        'Threads' = $sortedByThreads[$j].Process
        'Network' = $Network[$j].InstanceName
    }
}
$harray | Format-Table -AutoSize


CPU                    WS                      Handles                Threads                 Netwo
                                                                                              rk
---                    --                      -------                -------                 -----
msedge►17220           Memory Compression►2592 OneDrive►4220          System►4                code…
powershell_ise►10100   Code - Insiders►2860    System►4               Memory Compression►2592 code…
Grammarly.Desktop►7320 msedge►9840             explorer►8092          explorer►8092           code…
Code - Insiders►2860   dwm►5200                msedge►9840            msedge►9840             audi…
msedge►9840            pwsh►4464               lsass►908              SearchHost►12324        code…
Code - Insiders►12740  Grammarly.Desktop►7320  Grammarly.Desktop►7320 msedgewebview2►3268     dwm
OneDrive►4220          powershell_ise►10100    MonitoringHost►6568    Code - Insiders►16136   wind…
msed